In [2]:
import deepcut
from pathlib import Path
from pythainlp import word_tokenize
from matplotlib import pyplot as plt

import re
import pandas as pd
import numpy as np
from numpy import array
from numpy import argmax

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

from nltk import FreqDist, precision, recall, f_measure, NaiveBayesClassifier
from nltk.classify import apply_features
from nltk.classify import util
import collections, itertools
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [3]:
#data = pd.read_csv('train.csv')

In [4]:

texts = []
labels = []
with open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train.txt",encoding="utf8") as f:
    for line in f:
        texts.append(line.strip())
        
with open("C:/Users/60050204/BigData/wisesight-sentiment/kaggle-competition/train_label.txt",encoding="utf8") as f:
    for line in f:
        labels.append(line.strip())

df = pd.DataFrame({"category":labels, "texts":texts})
df.shape

(24063, 2)

In [5]:
df = df[df.category != 'q']

In [6]:
df.groupby('category').describe()

texts                       
          count unique        top freq
category                              
neg        6140   6129    #ERROR!   12
neu       13105  13087    #ERROR!   19
pos        4300   4300  สุ้ๆพี่ยศ    1

In [7]:
df['length'] = df['texts'].apply(word_tokenize).apply(len)
df['length'].describe()

count    23545.000000
mean        27.449310
std         45.216729
min          1.000000
25%          6.000000
50%         12.000000
75%         30.000000
max        758.000000
Name: length, dtype: float64

In [8]:
df

,category,texts,length
0,neu,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ,13
1,neu,คะ,1
2,neg,อิเหี้ยออมทำกูอยากกินเอ็มเค,9
3,neu,😅😅😅,1
4,neu,สวัสดีวันพุธ แนน อะไรนะ,8
...,...,...,...
24058,neg,แม่งควายล้วนนน,5
24059,neg,ดอยสุเทพน้องง ไปหมดแล้วววว #pm25,13
24060,neg,ค่าชุดอาจจะแพงกว่าส่วนลด,6
24061,neu,รัฐต้องการแค่ภาษีครับ,5


In [9]:
df.sample(100)

,category,texts,length
14792,neu,จะทันมาม่ะ,4
15816,neu,การที่ทูลกระหม่อมแทนตัวเองว่า 'ดิฉัน' แทนการใช...,42
15621,neu,ชิ้นส่วน B. หายไปครับ อยากเอารถ Honda CRV มาเข...,48
7713,neu,0.99% แค่ยาริสหรอคับ Fortuner ร่วมด้วยไหมคับ,14
628,neu,เมื่อพูดถึงสงกรานต์... มือก็อยากสาด ขาก็อยากแด...,106
...,...,...,...
13642,neu,ถ้าดูดในบ้านจะโดนยึดบ้านมั้ย,9
652,neg,ควรปรับปรุงในเรื่องคุณภาพของเนื้อนะคะ 2 วันก่อ...,94
19981,neg,ณ. เวลานี้ ทฤษฎีต่างๆมันช่าง ตีกันมะลุงตุงนัง ...,32
8343,neu,#คืนนี้ดื่มฟรี..รีบบอกต่อๆกันกับประสบการณ์ดีๆ@...,307


In [10]:
from gensim.models import Word2Vec 

In [11]:
y_train = [row for row in df['category']]
Y_train = array(y_train)
Y_train.shape

(23545,)

In [12]:
label_encoder = LabelEncoder()

In [13]:
integer_encoder = label_encoder.fit_transform(Y_train)

In [14]:
integer_encoder

array([1, 1, 0, ..., 0, 1, 1], dtype=int64)

In [15]:
ohe = OneHotEncoder(sparse=False)

In [16]:
onehot_encoded = ohe.fit_transform(integer_encoder.reshape(-1,1))

In [17]:
onehot_encoded

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.texts.values)

In [19]:
word_index = tokenizer.word_index

In [20]:
vocab_size = len(word_index)

In [21]:
vocab_size

85786

In [22]:
df['length'].describe()

count    23545.000000
mean        27.449310
std         45.216729
min          1.000000
25%          6.000000
50%         12.000000
75%         30.000000
max        758.000000
Name: length, dtype: float64

In [23]:
max_length = 758

In [111]:
x_train = tokenizer.texts_to_sequences(df.texts.values)

In [112]:
x_train_padded = pad_sequences(x_train, maxlen=max_length ,padding='post')

In [26]:
x_train_padded.shape

(23545, 758)

In [27]:
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout, Activation
from tensorflow.keras.models import Sequential

In [38]:
model = Sequential()
model.add(Embedding(vocab_size+1,32,input_length=max_length))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 758, 32)           2745184   
_________________________________________________________________
flatten_1 (Flatten)          (None, 24256)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3104896   
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 387       
Total params: 5,883,491
Trainable params: 5,883,491
Non-trainable params: 0
____________________________________________

In [39]:
from sklearn.model_selection import train_test_split

In [46]:
x_train, x_test, y_train, y_test = train_test_split(x_train_padded, onehot_encoded, test_size=0.2, shuffle=True)

In [41]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((16481, 758), (16481, 3), (7064, 758), (7064, 3))

In [42]:
x_train.shape

(16481, 758)

In [47]:
model.fit(x_train,y_train,
         batch_size=20,
         epochs=3,)

Train on 18836 samples
Epoch 1/3
18836/18836 [==============================] - 35s 2ms/sample - loss: 0.4470 - accuracy: 0.7972
Epoch 2/3
18836/18836 [==============================] - 34s 2ms/sample - loss: 0.1371 - accuracy: 0.9522
Epoch 3/3
18836/18836 [==============================] - 34s 2ms/sample - loss: 0.0235 - accuracy: 0.9927


In [49]:
test_loss,test_acc = model.evaluate(x_test,y_test)
print("\nTest accuracy:",test_acc*100,"%")
print("\nTest Loss:",test_loss*100,"%")


4709/4709 [==============================] - 1s 143us/sample - loss: 0.5825 - accuracy: 0.7927

Test accuracy: 79.27373051643372 %

Test Loss: 58.25328122382205 %


In [51]:
predicions = model.predict(x_test)
predicions

array([[2.2137975e-08, 9.9999762e-01, 2.3576858e-06],
       [9.9334466e-01, 1.4012678e-04, 6.5152990e-03],
       [7.8079691e-05, 9.8180765e-01, 1.8114313e-02],
       ...,
       [3.2126620e-02, 8.1406319e-01, 1.5381022e-01],
       [5.2216087e-02, 1.1206870e-01, 8.3571517e-01],
       [5.7857810e-04, 9.9314559e-01, 6.2758881e-03]], dtype=float32)

In [62]:
reverse_ohe = ohe.inverse_transform(predicions)
reverse_label = label_encoder.inverse_transform(reverse_ohe)
reverse_label
outputDataFrame = pd.DataFrame(reverse_label)
outputDataFrame

C:\Users\60050204\Anaconda3\envs\BigData\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


,0
0,neu
1,neg
2,neu
3,neu
4,pos
...,...
4704,neu
4705,neu
4706,neu
4707,pos


In [64]:
reverse_x_test = tokenizer.sequences_to_texts(x_test)
x_test_DF = pd.DataFrame(reverse_x_test)
x_test_DF

,0
0,แล้วแต่จะมองครับ เพราะต้องรอดูการปรับจูนหน้างา...
1,ตร พูดแบบนี้ผิด ม 157มั้ย
2,โหวว เซ็ตนี้ภาพสวยคมชัด โมเม้นท์ดีงามมากเรยค่ะ...
3,ได้ 4 เซียนทะลุรอบรองฯ โอ๊ต ระนอง ตัดกับ ตัวจี...
4,มีบริการทั้งหมด 4บริการ ครับ ชอบเป็นพิเศษก็คือ...
...,...
4704,จริงเหรอ 55555
4705,แอร์เอเชียเล็งสแกนใบหน้าเช็กอินเดินทาง
4706,พรุ่งนี้ เตรียมตังไว้😎
4707,กัฟเคยใช้เมเบอลีนเอาไม่อยู่เหมือนกันค่ะ😂 ตัวนี...


In [163]:
test_word = "ไอตู่ไอเหี้ย"
test_word_df = pd.DataFrame([test_word],columns=['texts'])
test_word_df

,texts
0,ไอตู่ไอเหี้ย


In [164]:
test_word_encoded = tokenizer.texts_to_sequences(test_word_df.texts.values)
test_word_padded = pad_sequences(test_word_encoded, maxlen=max_length ,padding='post')

In [165]:
pred = model.predict(test_word_padded)
pred

array([[0.03545961, 0.23003702, 0.7345034 ]], dtype=float32)

In [166]:
reverse_ohe = ohe.inverse_transform(pred)
reverse_label = label_encoder.inverse_transform(reverse_ohe)
reverse_label

C:\Users\60050204\Anaconda3\envs\BigData\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


array(['pos'], dtype='<U3')